In [ ]:
pip install botorch

In [2]:
import pandas as pd
import torch

train_data = pd.read_excel('data_sample_multiple_components_GLU_9objs.xlsx')
#print(train_data)
train_x = train_data.iloc[:,[0,1]].values
#print(train_X)
train_x = torch.tensor(train_x, dtype=torch.float32)
print(train_x)

train_obj = train_data.iloc[:,2:].values
#print(train_Y)
train_obj = torch.tensor(train_obj, dtype=torch.float32)
print(train_obj)

tensor([[0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000]])
tensor([[ 0.0450,  0.0500,  0.0090,  0.0190,  0.1520,  0.0000,  0.0210,  0.0270,
         -0.1990],
        [ 0.0390,  0.0540,  0.0130,  0.0640,  0.1800,  0.0000,  0.0260,  0.0210,
         -0.1990],
        [ 0.0470,  0.0660,  0.0140,  0.0280,  0.2280,  0.0000,  0.0330,  0.0400,
         -0.1990],
        [ 0.0430,  0.0570,  0.0120,  0.0370,  0.1870,  0.0000,  0.0270,  0.

In [4]:
from botorch import fit_gpytorch_model
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.transforms.outcome import Standardize
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.optim.optimize import optimize_acqf

NOISE_SE = torch.tensor([1,1,1,1,1,1,1,1,1])
#print(NOISE_SE)

# define models for objective and constraint
models = []
for i in range(train_obj.shape[-1]):
    train_y = train_obj[..., i:i+1]
    train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
    models.append(FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)))
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

BATCH_SIZE = 10
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0044, 0.3460],
        [0.0241, 0.1172],
        [0.0198, 0.5596],
        [0.0128, 2.3886],
        [0.0077, 2.9914],
        [0.0178, 0.6583],
        [0.0061, 2.6802],
        [0.0409, 0.8695],
        [0.0082, 0.6672],
        [0.0398, 2.5017]])


In [21]:
# predict y from the trained GP model given the X candidates
pred = model.posterior(candidates).mean.squeeze().detach().numpy()
print(pred)

[[ 0.04990618  0.05935005  0.01595806  0.02339904  0.20561391  0.01162659
   0.02295985  0.02496088 -0.32957563]
 [ 0.04991111  0.05934166  0.0159582   0.02338764  0.20571192  0.01162591
   0.02295913  0.02495979 -0.3526871 ]
 [ 0.04990536  0.05935494  0.015958    0.02340072  0.20563637  0.01162767
   0.02296001  0.02496025 -0.32335246]
 [ 0.04992073  0.05931995  0.01595852  0.02336579  0.20584401  0.01162286
   0.02295765  0.02495892 -0.4024646 ]
 [ 0.04991697  0.05933219  0.01595835  0.02337423  0.20583369  0.0116248
   0.02295828  0.02495838 -0.38029277]
 [ 0.04990766  0.05935437  0.01595802  0.02339535  0.20571938  0.0116278
   0.02295973  0.02495895 -0.3316051 ]
 [ 0.04991792  0.05932911  0.01595839  0.02337211  0.20583624  0.01162431
   0.02295813  0.02495852 -0.385933  ]
 [ 0.04991414  0.05935203  0.01595809  0.02338021  0.20594348  0.0116287
   0.0229589   0.02495551 -0.3557266 ]
 [ 0.04990793  0.05935417  0.01595801  0.02339479  0.20572776  0.01162757
   0.02295971  0.02495883